In [1]:
%load_ext autoreload
%autoreload 2

Dependencies

In [2]:
import pathlib as pb

# Environment
ROOT_PATH = pb.Path('..')
DATA_DIR_PATH = ROOT_PATH / 'data'
CACHE_DIR_PATH = ROOT_PATH / '.cache'
TRANSFORMERS_CACHE_DIR_PATH = CACHE_DIR_PATH / 'transformers'
DATASETS_CACHE_DIR_PATH = CACHE_DIR_PATH / 'datasets'
TEST_DATA_FILE = DATA_DIR_PATH / 'test_data.csv'
TRAIN_DATA_FILE = DATA_DIR_PATH / 'train_data.csv'

# Model Repositories
BERT_RO_MODEL_REPO = 'dumitrescustefan/bert-base-romanian-cased-v1'

In [3]:
import os

os.environ['TRANSFORMERS_CACHE'] = str(TRANSFORMERS_CACHE_DIR_PATH)
os.environ['HF_DATASETS_CACHE'] = str(DATASETS_CACHE_DIR_PATH)

In [4]:
from transformers import AutoTokenizer, AutoModel
import torch
import torchdata
import torchtext
from torch.utils.data import Subset, DataLoader, Dataset
from torch import backends
import typing
import pathlib as pb
import os
from typing import List, Tuple, Dict, Set
import random
import numpy as np
import pandas as pd
import platform
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import KFold

/home/invokariman/projects/git/nitro-lang-processing-2/.cache/pypoetry/virtualenvs/nitro-lang-processing-2-NQx-kwHa-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
from utils import get_available_device, read_data
from preprocess import BertPreprocessor
from data import SexismDataset
from models import BertFlatClassModel

Environment Setup

In [6]:
# Use available GPU
DEVICE: torch.device = get_available_device()

# Deterministic experiments
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
np.random.RandomState(SEED)
torch.manual_seed(SEED)
torch.use_deterministic_algorithms(True)

Dataset & Preprocessing

In [7]:
train_data_raw, test_data_raw = read_data(DATA_DIR_PATH)

In [8]:
train_dataset = SexismDataset(train_data_raw, BertPreprocessor(train_data_raw, BERT_RO_MODEL_REPO))
test_dataset = SexismDataset(test_data_raw, BertPreprocessor(test_data_raw, BERT_RO_MODEL_REPO))

Model Training

In [15]:
# Instantiate language model
model = BertFlatClassModel(BERT_RO_MODEL_REPO)

In [9]:
# Training specifications
batch_size = 8
num_workers = 6

In [10]:
# Inspired from: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html
kf = KFold(n_splits=5, shuffle=True, random_state=SEED)

for i, (train_idx, valid_idx) in enumerate(kf.split(train_dataset)):
    # Split the data
    train_subset = Subset(train_dataset, train_idx)
    valid_subset = Subset(train_dataset, valid_idx)

    # Create the dataloaders
    train_loader = DataLoader(
        train_subset,
        batch_size,
        pin_memory=True,
        num_workers=num_workers
    )
    valid_loader = DataLoader(
        train_subset,
        batch_size,
        pin_memory=True,
        num_workers=num_workers
    )